In [1]:
import gzip
import shutil
import os
import pandas as pd

# decompress LODES dataset(commuting data of NY)
def decompress_gzip(file_path, output_path):
    with gzip.open(file_path, 'rb') as f_in:
        with open(output_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    print(f"Decompressed {file_path} to {output_path}")

# Census Tabulation BlockCode (2020-)
file_name = 'ny_xwalk.csv.gz'
decompressed_file = 'ny_xwalk.csv'
decompress_gzip(file_name, decompressed_file)

# all job data with both workplace and residence located in NY(2021)
file_name2 = 'ny_od_main_JT00_2021.csv.gz'
decompressed_file2 = 'ny_od_main_JT00_2021.csv'
decompress_gzip(file_name2, decompressed_file2)

csv_file_path = 'ny_od_main_JT00_2021.csv'
df = pd.read_csv(csv_file_path)

csv_file_path = 'ny_xwalk.csv'
ny = pd.read_csv(csv_file_path)

Decompressed ny_xwalk.csv.gz to ny_xwalk.csv
Decompressed ny_od_main_JT00_2021.csv.gz to ny_od_main_JT00_2021.csv


<ipython-input-1-edb3664aae1c>:27: DtypeWarning: Columns (27,28,29,37) have mixed types. Specify dtype option on import or set low_memory=False.
  ny = pd.read_csv(csv_file_path)


In [2]:
ny.rename(columns={'tabblk2020': 'w_geocode'}, inplace=True)
ny = ny[['w_geocode','zcta']]
merged_df_left = pd.merge(df, ny, on='w_geocode', how='left')

ny.rename(columns={'w_geocode': 'h_geocode'}, inplace=True)
merged_df_left.rename(columns={'zcta': 'w_zcta'}, inplace=True)
merged_df = merged_df_left[['w_geocode','w_zcta','h_geocode','S000']]
merged_df = pd.merge(merged_df_left, ny, on='h_geocode', how='left')

df = merged_df.groupby(['w_zcta', 'zcta'])['S000'].sum().reset_index()

In [3]:
# merge ZIP Code Tabulation Area (ZCTA) Code with geographic location
filename = 'uszips.csv'
zip = pd.read_csv(filename)
zip = zip[['zip','lat','lng']]
zip.rename(columns={'zip': 'zcta'}, inplace=True)

df = pd.merge(df, zip, on='zcta', how='left')
df.rename(columns={'lat': 'h_lat','lng': 'h_lng'}, inplace=True)
zip.rename(columns={'zcta': 'w_zcta'}, inplace=True)
df = pd.merge(df, zip, on='w_zcta', how='left')
df.rename(columns={'lat': 'w_lat','lng': 'w_lng'}, inplace=True)

In [4]:
df = df.dropna(subset=['h_lat', 'h_lng', 'w_lat', 'w_lng'])
#print(df.isna().sum())

df['w_zcta'] = df['w_zcta'].astype(str).str.zfill(5)
df['zcta'] = df['zcta'].astype(str).str.zfill(5)

grouped_df = df.groupby(['zcta', 'w_zcta','h_lat', 'h_lng', 'w_lat', 'w_lng'], as_index=False)['S000'].sum()

w_zcta    0
zcta      0
S000      0
h_lat     0
h_lng     0
w_lat     0
w_lng     0
dtype: int64


In [5]:
import geopandas as gpd
from shapely.geometry import Point

# Neighborhood Tabulation Area（NTA） of NYC
csv_file_path = '2020_Neighborhood_Tabulation_Areas__NTAs__20240728.csv'
nta_df = pd.read_csv(csv_file_path)

locations_df = df
nta_gdf = gpd.GeoDataFrame(nta_df, geometry=gpd.GeoSeries.from_wkt(nta_df['the_geom']))

# home_work_GeoDataFrame
locations_df['home_point'] = locations_df.apply(lambda row: Point(row['h_lng'], row['h_lat']), axis=1)
locations_df['work_point'] = locations_df.apply(lambda row: Point(row['w_lng'], row['w_lat']), axis=1)

home_gdf = gpd.GeoDataFrame(locations_df, geometry='home_point')
work_gdf = gpd.GeoDataFrame(locations_df, geometry='work_point')

home_gdf.set_crs(epsg=4326, inplace=True)
work_gdf.set_crs(epsg=4326, inplace=True)
nta_gdf.set_crs(epsg=4326, inplace=True)

# home_NTA
home_gdf = gpd.sjoin(home_gdf, nta_gdf[['NTAName', 'geometry']], how='left', op='within')

# work_NTA
work_gdf = gpd.sjoin(work_gdf, nta_gdf[['NTAName', 'geometry']], how='left', op='within')


result_df = locations_df.copy()
result_df['home_NTAName'] = home_gdf['NTAName']
result_df['work_NTAName'] = work_gdf['NTAName']

result_df.drop(columns=['home_point', 'work_point'], inplace=True)

#result_df.to_csv('matched_data.csv', index=False)

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
result_df.dropna(inplace=True)

# group commuters by NTA
aggregated_df = result_df.groupby(['home_NTAName', 'work_NTAName']).agg({
    'S000': 'sum'
}).reset_index()

df_commute  = aggregated_df
#df_commute  = df_aggregated

In [9]:
import pandas as pd
import geopandas as gpd
import folium
from shapely import wkt
from folium import Popup

nta_gdf = gpd.GeoDataFrame(nta_df, geometry=gpd.GeoSeries.from_wkt(nta_df['the_geom']))

m = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

# add neighborhood boundaries
for _, row in nta_gdf.iterrows():
    folium.GeoJson(
        row['geometry'],
        name=row['NTAName'],
        tooltip=row['NTAName'],
        style_function=lambda feature: {
            'color': 'black',
            'weight': 2,
            'fillOpacity': 0,
        }
    ).add_to(m)

# commuters layer
layer_1000 = folium.FeatureGroup(name='commuters > 1000', show=True)
layer_500 = folium.FeatureGroup(name='commuters 500 - 1000', show=True)
layer_300 = folium.FeatureGroup(name='commuters 300 - 500', show=True)
layer_100 = folium.FeatureGroup(name='commuters 100 - 300', show=True)
layer_0 = folium.FeatureGroup(name='commuters 0 - 100', show=True)

# commuting routes
for _, row in df_commute.iterrows():
    home_geom = nta_gdf[nta_gdf['NTAName'] == row['home_NTAName']]['geometry'].values[0]
    work_geom = nta_gdf[nta_gdf['NTAName'] == row['work_NTAName']]['geometry'].values[0]
    home_centroid = home_geom.centroid
    work_centroid = work_geom.centroid

    # set color and weight based on number of commuters
    if row['S000'] > 1000:
        color = 'red'
        weight = 6
        layer = layer_1000
    elif row['S000'] > 500:
        color = 'orange'
        weight = 4
        layer = layer_500
    elif row['S000'] > 300:
        color = 'green'
        weight = 2
        layer = layer_300
    elif row['S000'] > 100:
        color = 'rgb(057, 081, 162)'
        weight = 1
        layer = layer_100
    else:
        color = 'rgb(202, 232, 242)'
        weight = 1
        layer = layer_0

    polyline = folium.PolyLine(
        locations=[(home_centroid.y, home_centroid.x), (work_centroid.y, work_centroid.x)],
        color=color,
        weight=weight,
        opacity=0.6
    )

    # add click marker
    popup = Popup(f"Home: {row['home_NTAName']}<br>Work: {row['work_NTAName']}<br>Commute: {row['S000']}", max_width=300)
    polyline.add_child(popup)
    layer.add_child(polyline)

# add layers
layer_1000.add_to(m)
layer_500.add_to(m)
layer_300.add_to(m)
layer_100.add_to(m)
layer_0.add_to(m)

folium.LayerControl().add_to(m)

m.save('commute_map.html')


In [14]:
from google.colab import files
files.download('commute_map.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>